In [1]:
import os
import sys
import subprocess
import datetime as dt
import itertools
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
import dask

In [2]:
cluster = LocalCluster(n_workers=4)
client = Client(cluster)

In [3]:
# define leadtimes to extract (as indices to daily files, so 0 is 1/2 day lead)
def agg_l(xl):
    for x in xl:
        if x<30:
            yield x
        elif x<90 and x%15==0:
            yield x
        elif x%30==0:
            yield x

leads=[el for el in agg_l(range(0,215))]
print(' '.join(str(el) for el in leads))
print(len(leads))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 45 60 75 90 120 150 180 210
39


In [4]:
mdir='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5'
#fnameCanESMjoined=lambda mdir, yyyy, mm, dd, hh:f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_6hr_surface_tso.nc"
fnameCanESMdaily=lambda mdir, yyyy, mm, dd, hh:f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_1d_surface_tso.nc"
fnameCanESMAnom=lambda mdir, lfirst, llast, mm, dd, hh:f"{mdir}/joined/anom/anom_cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}0100_L_{lfirst}_{llast}_ocean_1d_surface_tso.nc"

In [5]:
print('test',)

test


In [6]:
#for mm in range(1,13): month loop
mm=1
flist=[fnameCanESMdaily(mdir,yyyy,mm,1,0) for yyyy in range(1993,2025) if yyyy<2024 or mm<=6] # stop at Jul 2024


In [7]:
ff=xr.open_mfdataset(flist,parallel=True,combine='nested',concat_dim='reftime',chunks={'reftime':-1,'leadtime':5,'r':-1,'lat':-1,'lon':-1},decode_times=False)

/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc3/lib/python3.9/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "leadtime" starting at index 5. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc3/lib/python3.9/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "leadtime" starting at index 5. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc3/lib/python3.9/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "leadtime" starting at index 5. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc3/lib/python3.9/site-pac

In [8]:
ff

<xarray.Dataset> Size: 71GB
Dimensions:   (reftime: 32, lat: 180, bnds: 2, lon: 360, r: 20, leadtime: 215)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) float64 256B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    time      (leadtime) float64 2kB dask.array<chunksize=(5,), meta=np.ndarray>
Dimensions without coordinates: bnds, r
Data variables:
    lat_bnds  (reftime, lat, bnds) float64 92kB dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
    lon_bnds  (reftime, lon, bnds) float64 184kB dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    tso       (reftime, r, leadtime, lat, lon) float64 71GB dask.array<chunksize=(1, 20, 5, 180, 360), meta=np.ndarray>
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                  Sat Aug 31 04:30:30 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

In [9]:
ff.tso

<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 215, lat: 180, lon: 360)> Size: 71GB
dask.array<concatenate, shape=(32, 20, 215, 180, 360), dtype=float64, chunksize=(1, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) float64 256B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    time      (leadtime) float64 2kB dask.array<chunksize=(5,), meta=np.ndarray>
Dimensions without coordinates: r
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

In [10]:
EClim=ff.tso.mean(dim='r').mean(dim='reftime')

In [11]:
EClim

<xarray.DataArray 'tso' (leadtime: 215, lat: 180, lon: 360)> Size: 111MB
dask.array<mean_agg-aggregate, shape=(215, 180, 360), dtype=float64, chunksize=(5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    time      (leadtime) float64 2kB dask.array<chunksize=(5,), meta=np.ndarray>

In [12]:
Anom0=ff.tso-EClim

In [13]:
66.43/43

1.5448837209302326

In [14]:
Anom0

<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 215, lat: 180, lon: 360)> Size: 71GB
dask.array<sub, shape=(32, 20, 215, 180, 360), dtype=float64, chunksize=(1, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) float64 256B 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    time      (leadtime) float64 2kB dask.array<chunksize=(5,), meta=np.ndarray>
Dimensions without coordinates: r

In [15]:
int(len(ff.leadtime.values)/5)

43

In [16]:
p0='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined'

In [17]:
dsout=xr.Dataset(data_vars={'sst_anom':Anom0,},
                coords=dict(leadtime=ff.leadtime,
                            reftime=ff.reftime,
                            lat=ff.lat,
                            lon=ff.lon))

In [ ]:
dsout

In [ ]:
datasets=[]
paths=[]
for ix in range(0,int(len(ff.leadtime.values)/5)):
    fname=f"test2_l_{5*ix:03}_{5*(ix+1)-1:03}.nc"
    temp=dsout.isel(leadtime=slice(5*ix,5*(ix+1)))
    paths.append(os.path.join(p0,fname))
    datasets.append(temp)
    #temp.to_netcdf(os.path.join(p0,fname),mode='w')

In [ ]:
%%time
out=xr.save_mfdataset(datasets,paths,mode='w')

In [ ]:
client.shutdown()
cluster.close()

In [18]:
%%time
for ix in range(0,int(len(ff.leadtime.values)/5)):
    print(ix)
    fname=f"test2_l_{5*ix:02}_{5*(ix+1)-1:02}.nc"
    temp=Anom0.isel(leadtime=slice(5*ix,5*(ix+1)))
    temp.to_netcdf(os.path.join(p0,fname),mode='w')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


Exception ignored in: <function CachingFileManager.__del__ at 0x14fe4f6fe160>
Traceback (most recent call last):
  File "/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc3/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc3/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2618, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2585, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2113, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID


16
17
18
19
20
21
22
23
24


Exception ignored in: <function CachingFileManager.__del__ at 0x14fe4f6fe160>
Traceback (most recent call last):
  File "/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc3/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/home/ords/crd/ccrn/reo000/miniconda3/envs/py39dc3/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2618, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2585, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2113, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: Not a valid ID
HDF5-DIAG: Error detected in HDF5 (1.14.3) thread 1:
  #000: H5A.c line 1891 in H5Aiterate2(): invalid location identifier
    major: Invalid arguments to routine
    minor: Inappropriate type
  #001: H5VLint.c line 1741 in H5VL_vol_object(): invalid identifier type to fu

RuntimeError: NetCDF: Can't open HDF5 attribute

In [ ]:
client.shutdown()
cluster.close()

In [ ]:
# check daily file:
p0='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined'
fn='cwao_CanESM5.1p1bc-v20240611_hindcast_S2024060100_ocean_1d_surface_tso.nc'
ff=xr.open_dataset(os.path.join(p0,fn))

In [ ]:
ff

In [ ]:
ff.reftime.values

In [ ]:
EM=ff.mean(dim='r')

In [ ]:
EM.tso